<a href="https://colab.research.google.com/github/lathashree01/NLP_implementations/blob/sequence_classification/Sequence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import pandas as pd
import re
from tqdm.auto import tqdm

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from datasets import Dataset, load_dataset
from transformers import get_scheduler
import evaluate

from sklearn.model_selection import train_test_split

import torch
# from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [ ]:
!pip install -U datasets evaluate

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words("english"))
punct_re = re.compile(r'[^\w\s]')

In [ ]:
# Dummy data
data = {
    "text": [
        "I loved the movie! It was fantastic.",
        "The film was too long and boring.",
        "Excellent performances by the cast.",
        "Terrible script and poor acting.",
        "I loved the movie! It was fantastic.",
        "The film was too long and boring.",
        "Excellent performances by the cast.",
        "Terrible script and poor acting."
    ],
    "label": [1, 0, 1, 0, 1, 0, 1, 0]
}

In [ ]:
df = pd.DataFrame(data)
df

,text,label
0,I loved the movie! It was fantastic.,1
1,The film was too long and boring.,0
2,Excellent performances by the cast.,1
3,Terrible script and poor acting.,0
4,I loved the movie! It was fantastic.,1
5,The film was too long and boring.,0
6,Excellent performances by the cast.,1
7,Terrible script and poor acting.,0


### Preprocess data

In [ ]:
def preprocess_text(text):
  words = word_tokenize(text.lower())
  filtered = []
  for w in words:
    clean = punct_re.sub("", w)
    if clean and clean not in stop_words:
      filtered.append(clean)
  return " ".join(filtered)


In [ ]:
# preprocess_text(df.iloc[0].text)

In [ ]:
df["text"] = df["text"].apply(preprocess_text)
df.head()

,text,label
0,loved movie fantastic,1
1,film long boring,0
2,excellent performances cast,1
3,terrible script poor acting,0
4,loved movie fantastic,1


In [ ]:
train_df, val_df = train_test_split(df[["text", "label"]], test_size=0.2, random_state=42, stratify=df["label"])

### Load huggingface model and tokenizer

In [ ]:
model_id = "google-bert/bert-base-uncased"

In [ ]:
bert_tokeniser = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load dataset

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
def tokenize_function(example):
    return bert_tokeniser(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=bert_tokeniser)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Training setup

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [ ]:
metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8480392156862745, 'f1': 0.89419795221843}

In [ ]:
raw_datasets['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
# data_collator = DataCollatorWithPadding(tokenizer=bert_tokeniser)
# train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)

In [ ]:
# train_dataset = train_dataset.rename_column("label", "labels")
# val_dataset = val_dataset.rename_column("label", "labels")

# train_dataset = train_dataset.remove_columns(["text","__index_level_0__"])
# val_dataset = val_dataset.remove_columns(["text","__index_level_0__"])

In [ ]:
# def tokenize_function(example):
#     return bert_tokeniser(example["text"], truncation=True)

# train_dataset = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# train_dataloader = DataLoader(
#     train_dataset, shuffle=True, batch_size=2, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     val_dataset, batch_size=2, collate_fn=data_collator
# )

In [ ]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}